In [2]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel('../data/帕累托分析.xlsx')
df.head()

,店名,店风格,店性质,品类描述,品牌描述,所属小区,所属小区.1,销售日期,销售额,毛利
0,成都店,时尚馆,自有店,女士轻便服装,WHO.A.U,西南,西南,2017-09-30,1046,209
1,成都店,时尚馆,自有店,女士轻便服装,WHO.A.U,西南,西南,2017-09-30,3366,673
2,北京一期（百货）,生活馆,自有店,女士上班服装,X.ZHINING,华北,华北,2017-09-30,3979,124
3,重庆店,时尚馆,自有店,女士上班服装,WHO.A.U,西南,西南,2017-09-30,5960,894
4,重庆店,时尚馆,自有店,女士上班服装,WHO.A.U,西南,西南,2017-09-30,1008,151


In [36]:
# 以商品种类进行分组计算销售额
k_m = df.groupby('品类描述').sum()['销售额'].reset_index()
# 按销售额降序排序
k_m = k_m.sort_values(by=['销售额'],ascending=False)
# 计算总销售额
k_m['总销售额'] = k_m['销售额'].sum()
# 计算销售额占比
k_m['销售额占比'] = k_m['销售额'] / k_m['总销售额']
# 累计销售额占比
zhanbi = list(k_m['销售额占比'])
neiji = []
for i in range(len(zhanbi)):
    if i == 0:
        neiji.append(zhanbi[i])
    else:
        neiji.append(neiji[i-1]+zhanbi[i])
k_m['累计占比'] = neiji
# 判断累加值是否大于 80%
k_m['flag'] = k_m['累计占比'].map(lambda x: 1 if x < 0.8 else 0)
k_m

,品类描述,销售额,总销售额,销售额占比,累计占比,flag
15,男士配饰,14453957,58782764,0.245888,0.245888,1
16,运动服装及用品,10311949,58782764,0.175425,0.421312,1
0,儿童服装,9866262,58782764,0.167843,0.589155,1
6,女士皮鞋,5692346,58782764,0.096837,0.685992,1
11,床上用品,4181450,58782764,0.071134,0.757126,1
3,女士內衣,2183940,58782764,0.037153,0.794279,1
2,女士上班服装,2004766,58782764,0.034105,0.828383,0
7,女士轻便服装,1571104,58782764,0.026727,0.855111,0
13,男士皮鞋,1416877,58782764,0.024104,0.879214,0
14,男士轻便服装,1221876,58782764,0.020786,0.900001,0


In [44]:
# 绘制帕累托图
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
from pyecharts.commons.utils import JsCode


# 选出占比达到 80% 的销售额
m = list(k_m[k_m['flag'] == 1]['销售额'])[-1]

x_data = list(k_m['品类描述'])

color_function = """
        function (params) {
            if (params.value >= %s) {
                return '#749f83';
            } else  {
                return '#d48265';
            }
        }
        """ % str(m)
bar = (
    Bar()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="销售额",
        y_axis=list(k_m['销售额']),
        z=0,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(color=JsCode(color_function))
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="销售额占比",
            type_="value",
            min_=0,
            max_=1.1,
            #interval=5,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True, trigger="axis", axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="销售额",
            type_="value",
            #min_=0,
            #max_=250,
            #interval=50,
            axislabel_opts=opts.LabelOpts(formatter="{value} 元"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        legend_opts=opts.LegendOpts(
            type_="scroll",
        )
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="销售额累计占比",
        yaxis_index=1,
        y_axis=list(k_m['累计占比']),
        label_opts=opts.LabelOpts(is_show=False),
        z=1,
        markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(y=0.8)])
    )
)

bar.overlap(line).render('../output/帕累托图.html')
bar.overlap(line).render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
